# Manipulating data and metadata in cf-python (version 2)

## Homepage: https://cfpython.bitbucket.io
## Online, searchable documenation: https://cfpython.bitbucket.io/docs/latest

## Contents:

### 1. Read, inspect, write netCDF files
### 2. Subspace
### 3. Data
### 4. Calculate statistics
### 5. Other file formats

In [ ]:
import cf
cf.__version__

# 1. Read, inspect and write files
https://cfpython.bitbucket.io/docs/latest/generated/cf.read.html

In [ ]:
f = cf.read('ncas_data/IPSL-CM5A-LR_r1i1p1_tas_n96_rcp45_mnth.nc')[0]

In [ ]:
f

In [ ]:
print f

In [ ]:
f.dump()

### Properties

In [ ]:
f.properties()

In [ ]:
f.getprop('project_id')

In [ ]:
f.setprop('project_id', 'banana')
f.getprop('project_id')

In [ ]:
f.delprop('project_id')
f.getprop('project_id')

In [ ]:
f.getprop('project_id', 'UNSET')

### Shorthand for named CF properties
http://cfconventions.org/Data/cf-conventions/cf-conventions-1.7/cf-conventions.html#attribute-appendix

In [ ]:
print f.standard_name
f.standard_name = 'banana'
print f.standard_name
del f.standard_name
f.standard_name = 'air_temperature'
print f.standard_name

### Reading many files

In [ ]:
fl = cf.read('ncas_data/data[2-7].nc')
fl

In [ ]:
for x in fl:
    print 'NAME:', x.name(), 'SHAPE:', x.shape, 'UNITS:', x.units

### Select by list position

In [ ]:
g = fl[0]
g

In [ ]:
fl[4:]

### Select by metadata
https://cfpython.bitbucket.io/docs/latest/generated/cf.FieldList.select.html

In [ ]:
fl.select('air_temperature')

In [ ]:
help(fl.select)

In [ ]:
fl.select('northward_wind')

In [ ]:
fl.select({'units': 'km h-1'})

In [ ]:
fl.select('(east|north)ward_wind')

# Write fields to a netCDF file
https://cfpython.bitbucket.io/docs/latest/generated/cf.write.html

In [ ]:
cf.write(f, 'new_file.nc')

In [ ]:
g = cf.read('new_file.nc')[0]
f.equals(g)

# 2. Subspace a field

https://cfpython.bitbucket.io/docs/latest/generated/cf.Field.subspace.html

### Index-space: [square brackets]

In [ ]:
f

In [ ]:
print f.subspace[0, 0, 0]

In [ ]:
f[0, 0, 0] # shorthand method - leave out the .subspace

In [ ]:
f.subspace[0:6, :, :]

### Metadata-space: (round brackets)

In [ ]:
print f

In [ ]:
print f.subspace(longitude=180) # No shorthand for the "round brackets" form

#### `cf.lt(30)` is a "query" that means *less than 30*

https://cfpython.bitbucket.io/docs/latest/function.html#comparison

In [ ]:
print f.subspace(latitude=cf.lt(30)) 

#### `cf.wi(90, 270)` is a query that means *within the range [90, 270]*

In [ ]:
print f.subspace(longitude=cf.wi(90, 270))

In [ ]:
g = f.subspace(time=cf.dt('1965-11-16'))
print g

In [ ]:
# In-line images
%matplotlib inline
# Turn off warnings
import warnings
warnings.filterwarnings("ignore")

import cfplot as cfp
cfp.con(g)

#### `T` is shorthand for *time*

In [ ]:
print f.subspace(T=cf.ge(cf.dt('1967-2-18')))

In [ ]:
print f.subspace(T=cf.month(4))

In [ ]:
print f.subspace(time=cf.dt('1965-11-16'), Y=cf.gt(30))

# 3. The field's data

In [ ]:
f.data

#### Get the data as a `numpy` array

In [ ]:
f.array # This is numpy array

In [ ]:
print type(f.array)
f.array[-1, 2, -3]

In [ ]:
f.subspace[-1, 2, -3].array

In [ ]:
x = f.copy()
x.subspace[-1, -1, -1] = -999
x.subspace[-1, -1, -1].array

In [ ]:
x.subspace[-1, ...] = 888
x.subspace[-1, ...].array

In [ ]:
x.subspace[0, ...] = x.subspace[-1, ...] - 111
x.subspace[0, ...].array

### Modify the data where a condition is met
https://cfpython.bitbucket.io/docs/latest/generated/cf.Field.where.html

In [ ]:
f.min(), f.mean(), f.max()

#### Set values below 290 to missing data

In [ ]:
x = f.where(cf.lt(290), cf.masked)
print x.min(), x.mean(), x.max()
cfp.con(x.subspace[0])

### Manipulate the axes

In [ ]:
f.transpose(['X', 'T', 'Y'])

### Modifying the units

In [ ]:
f = cf.read('ncas_data/IPSL-CM5A-LR_r1i1p1_tas_n96_rcp45_mnth.nc')[0]
f.units, f.mean()

In [ ]:
f.units = 'degC'
f.units, f.mean()

In [ ]:
f.Units # Upper case "U" gives a units object that we can manipulate

In [ ]:
f.Units += 273.15
f.Units, f.units, f.mean()

### Field arithmetic

In [ ]:
f

In [ ]:
f.min(), f.mean(), f.max()

In [ ]:
g = f + 2
g

In [ ]:
g.min(), g.mean(), g.max()

In [ ]:
g = f - f
g

In [ ]:
g.min(), g.mean(), g.max()

In [ ]:
x = f.copy()
x.units = 'degC'
x.data

#### Subtract the celcius field from the Kelvin field and check that the result is zero 

In [ ]:
(f - x).mean()

In [ ]:
g = f * f
g

#### Find the anomalies relaitive to the first time

In [ ]:
first_time = f.subspace[0]
first_time = first_time.transpose(['Y', 'T', 'X'])
first_time

In [ ]:
g  = f - first_time
g

In [ ]:
g.min(), g.mean(), g.max()

# 4. Statistical operations
https://cfpython.bitbucket.io/docs/latest/generated/cf.Field.collapse.html

In [ ]:
g = f.collapse('max')
g

In [ ]:
g.data

In [ ]:
g = f.collapse('T: mean')
print g
print 'data values:\n', g.data
print '\ntime bounds:\n', g.coord('T').bounds.dtarray

#### Collapse multiple axes simultaneously

In [ ]:
g = f.collapse('X: Y: sd')
g

#### Collapse an axis into groups, rather than a single value

In [ ]:
g = f.collapse('T: mean', group=cf.seasons())
print g

#### `cf.seasons()` is a list of queries, each of which defines a range of months

In [ ]:
cf.seasons()

#### By default, collpases are **not** weighted

In [ ]:
g = f.collapse('area: mean', weights='area') # Area mean for each time
g = g.collapse('T: max')                     # Time maxiumum of the area means
g.data
print g

### File aggregation

#### Create a sequence of files on disk, each of which contains one year

In [ ]:
f = cf.read('ncas_data/IPSL-CM5A-LR_r1i1p1_tas_n96_rcp45_mnth.nc')[0]
print f
for i in range(10):
    g = f.subspace[12*i:12*(i+1)]
    year = g.coord('T').year.array[0]
    new_file = 'air_temperature_'+str(year)+'.nc'
    cf.write(g, new_file)
    print '    ',new_file

#### In ipython `!` preceeeds a shell command

In [ ]:
!ls -o air_temperature_*.nc

In [ ]:
f2 = cf.read('air_temperature_*.nc')
print f2

In [ ]:
f.equals(f2[0])

In [ ]:
f3 = cf.read('air_temperature_*.nc', aggregate=False)
f3

# 5. PP and UM files

In [ ]:
x = cf.read('ncas_data/aaaaoa.pmh8dec.pp')
x

In [ ]:
print x[1]

In [ ]:
cf.write(x, 'aaaaoa.pmh8dec.nc')